In [5]:
import numpy as np
import pickle
import time
import copy

### Pickling

In [6]:
def read_pickle(filename):
    infile = open(filename,'rb')
    dict_data = pickle.load(infile)
    infile.close()
    return dict_data

In [7]:
def write_pickle(dict_data, filename):
    pickling_on = open(filename+'.pickle',"wb")
    pickle.dump(dict_data, pickling_on)
    pickling_on.close()
    print('saved file')

### Tasksets

In [8]:
# CAN CHANGE INPUT FILE NAME HERE

dict_data = read_pickle('data/experiment_data.pickle')

print('dict_data keys: ', dict_data.keys())
print()
print('num of tasksets keys: ', len(dict_data['tasksets'].keys()))
print()
print('num of results keys: ', len(dict_data['results'].keys()))
print()
print('results[0] keys: ', dict_data['results'][0].keys())
print()
print('other_test_data keys: ', dict_data['other_test_data'].keys())

tasksets = dict_data['tasksets']
results = dict_data['results']
other_test_data = dict_data['other_test_data']

dict_data keys:  dict_keys(['tasksets', 'results', 'other_test_data'])

num of tasksets keys:  12000

num of results keys:  12000

results[0] keys:  dict_keys(['percent_comp_int', 'percent_bdwth_int', 'num_tasks'])

other_test_data keys:  dict_keys(['V_k', 'U_j', 'C_k', 'B_j', 'delta_jk', 'alpha', 'beta', 'cloud_server', 'edge_server', 'unit_b', 'unit_c'])


### Heuristic - New Gamma

In [9]:
def calculate_split_values(tasks, theta_i, S_i, tau_i, N_E_i, edge_server, cloud_server, C_k, B_j):
    
    gamma_ijk = {}
    
    N_I_e = {}
    for edge in edge_server:
        N_I_e[edge] = []

        for task in N_E_i:
            if edge in N_E_i[task]:
                N_I_e[edge].append(task)
    
    for task in tasks:
        gamma_ijk[task] = {}
        
        for edge in N_E_i[task]:
            gamma_ijk[task][edge] = {}
    
            for proc_server in edge_server+cloud_server:
            
                numer = (S_i[task] / tau_i[task]) / B_j[edge]
                denom = (theta_i[task] / tau_i[task]) / C_k[proc_server]
                
                gamma_ijk[task][edge][proc_server] = numer / (numer + denom)
                
                
    return gamma_ijk

In [10]:
def calculate_utilization_values(tasks, theta_i, S_i, tau_i, gamma_ijk, N_E_i, delta_jk):
    
    utilizations = {}
    
    for task in tasks:
        utilizations[task] = {}
        for edge in N_E_i[task]:
            for jk in delta_jk:
                if edge == jk[0]:
                    time1 = (tau_i[task] - 2*delta_jk[jk]) * gamma_ijk[task][edge][jk[1]]
                    time2 = (tau_i[task] - 2*delta_jk[jk]) * (1-gamma_ijk[task][edge][jk[1]])
                    
                    # (bandwidth util, compute util)
                    utilizations[task][jk] = ((S_i[task]/time1), (theta_i[task]/time2))
    
    return utilizations

In [11]:
def calculate_priorities(utilizations, G_i, C_k, B_j):
    p_ijk = {}
    
    for task in utilizations.keys():
        for jk in utilizations[task].keys():
            
            p_ijk[(task, jk[0], jk[1])] = G_i[task]/(utilizations[task][jk][0]/B_j[jk[0]] * utilizations[task][jk][1]/C_k[jk[1]])
            
    return p_ijk

In [12]:
def heuristic_no_update(tasks, theta_i, S_i, tau_i, G_i, N_E_i, C_k, B_j, delta_jk, edge_server, cloud_server):
    '''
    IMPORTANT: remember to input copy of B_j and C_k since they will be changed 
    '''
        
    total_profit = 0
    num_tasks_offloaded = 0
    
    gamma_ijk = calculate_split_values(tasks, theta_i, S_i, tau_i, N_E_i, edge_server, cloud_server, C_k, B_j)
    utilizations = calculate_utilization_values(tasks, theta_i, S_i, tau_i, gamma_ijk, N_E_i, delta_jk)
    p_ijk = calculate_priorities(utilizations, G_i, C_k, B_j)
    
    num_runs = 0
    
    while p_ijk != {}:
        
        max_key = max(p_ijk, key= lambda x:p_ijk[x])
        
        if utilizations[max_key[0]][(max_key[1], max_key[2])][0] <= B_j[max_key[1]] and utilizations[max_key[0]][(max_key[1], max_key[2])][1] <= C_k[max_key[2]]:
            
            B_j[max_key[1]] -= utilizations[max_key[0]][(max_key[1], max_key[2])][0]
            C_k[max_key[2]] -= utilizations[max_key[0]][(max_key[1], max_key[2])][1]

            total_profit += G_i[max_key[0]]
            num_tasks_offloaded += 1
            
            keys_to_pop = []
            for key in p_ijk.keys():
                if key[0] == max_key[0]:
                    keys_to_pop.append(key)
            for key in keys_to_pop:
                p_ijk.pop(key)
        
        else:
            p_ijk.pop(max_key)
            
    
    return total_profit, num_tasks_offloaded

In [13]:
def check_runtime_and_profit_heusristic_no_update(tasksets, other_test_data, taskset_ID):
    
    taskset = tasksets[taskset_ID]
    
    tasks = taskset['tasks']
    G_i = taskset['G_i']
    tau_i = taskset['tau_i']
    theta_i = taskset['theta_i']
    S_i = taskset['S_i']
    N_E_i = taskset['N_E_i']
    
    V_k = other_test_data['V_k']
    U_j = other_test_data['U_j']
    C_k = other_test_data['C_k']
    B_j = other_test_data['B_j']
    delta_jk = other_test_data['delta_jk']
    alpha = other_test_data['alpha']
    beta = other_test_data['beta']
    cloud_server = other_test_data['cloud_server']
    edge_server = other_test_data['edge_server']
    b_unit = other_test_data['unit_b']
    c_unit = other_test_data['unit_c']
    
#     tasks_copy = tasks.copy()
#     N_E_i_copy = copy.deepcopy(N_E_i)
    B_j_copy = B_j.copy()
    C_k_copy = C_k.copy()
    
    start_time = time.time()
    total_profit, num_tasks_offloaded= heuristic_no_update(tasks, theta_i, S_i, tau_i, G_i, N_E_i, C_k_copy, B_j_copy, delta_jk, edge_server, cloud_server)
    total_time_execution = time.time() - start_time
    
    return total_profit, num_tasks_offloaded, total_time_execution



### Driver Code

In [14]:
def main():
    
#     ******************************
#           CHANGE INPUT FILE
#     ******************************
    
    dict_data = read_pickle('data/experiment_data.pickle')
    tasksets = dict_data['tasksets']
    results = dict_data['results']
    other_test_data = dict_data['other_test_data']
    print('Information from main()')
    print('-----------------------------------------------')
    print('Number of tasksets: ', len(tasksets.keys()))
    
    heuristic_profit, heuristic_runtime, heuristic_num_offloaded = {}, {}, {}
    
    
    number_evaluated = 0
    
    for ID in tasksets.keys():
        
        profit, num_tasks_offloaded, total_time_execution = check_runtime_and_profit_heusristic_no_update(tasksets, other_test_data, ID)

        heuristic_profit[ID] = profit
        heuristic_num_offloaded[ID] = num_tasks_offloaded
        heuristic_runtime[ID] = total_time_execution

        number_evaluated += 1
        if(number_evaluated%1000 == 0):
            print('number of tasksets evaluated: ', number_evaluated)
            
    print('-----------------------------------------------')
    print()
    print()
    
    dict_data['heuristic_profit'] = heuristic_profit
    dict_data['heuristic_num_offloaded'] = heuristic_num_offloaded
    dict_data['heuristic_runtime'] = heuristic_runtime
    
#     ****************************************
#             CHANGE OUTPUT FILE NAME 
#     ****************************************

    write_pickle(dict_data, 'data/heuritic_results_new_gamma_2_full')
    
    
    
#     ****************************************
#                 CHANGE SPLITTING
#     ****************************************
    sizes = [40, 60, 80, 100, 120]
    split_data = {}

    for size in sizes:
        split_data[size] = {'tasksets':{}, 'results':{},'heuristic_profit':{},'heuristic_runtime':{}, 'heuristic_num_offloaded':{}}
        split_data[size]['other_test_data'] = dict_data['other_test_data']
    
    for ID in dict_data['tasksets'].keys():
        num_tasks = len(dict_data['tasksets'][ID]['tasks'])
        if num_tasks in sizes:
            split_data[num_tasks]['tasksets'][ID] = dict_data['tasksets'][ID]
            split_data[num_tasks]['results'][ID] = dict_data['results'][ID]
            split_data[num_tasks]['heuristic_profit'][ID] = dict_data['heuristic_profit'][ID]
            split_data[num_tasks]['heuristic_runtime'][ID] = dict_data['heuristic_runtime'][ID]
            split_data[num_tasks]['heuristic_num_offloaded'][ID] = dict_data['heuristic_num_offloaded'][ID]
        else:
            print("Wrong instance size: [ID, size] = [", ID, ", ", num_tasks, "]")
            break

    print()
    for size in sizes:
        print('Number of tasksets with ', size, ' tasks: ', len(split_data[size]['tasksets'].keys()))
    print()
    
    
#     ****************************************
#             CHANGE OUTPUT FILES' NAMES
#     ****************************************
    for size in sizes:
        path = "data/heuristic_results_size_" + str(size)
        write_pickle(split_data[size], path)

    return split_data


In [15]:
start_time_overall = time.time()
split_data = main()

Information from main()
-----------------------------------------------
Number of tasksets:  12000
number of tasksets evaluated:  1000
number of tasksets evaluated:  2000
number of tasksets evaluated:  3000
number of tasksets evaluated:  4000
number of tasksets evaluated:  5000
number of tasksets evaluated:  6000
number of tasksets evaluated:  7000
number of tasksets evaluated:  8000
number of tasksets evaluated:  9000
number of tasksets evaluated:  10000
number of tasksets evaluated:  11000
number of tasksets evaluated:  12000
-----------------------------------------------


saved file

Number of tasksets with  40  tasks:  2400
Number of tasksets with  60  tasks:  2400
Number of tasksets with  80  tasks:  2400
Number of tasksets with  100  tasks:  2400
Number of tasksets with  120  tasks:  2400

saved file
saved file
saved file
saved file
saved file


In [16]:
sizes = [40, 60, 80, 100, 120]
for size in sizes:
    subset_heuristic_run_time = list(split_data[size]['heuristic_runtime'].values())
    print(size, ' mean time limit: ', sum(subset_heuristic_run_time)/len(subset_heuristic_run_time))
    subset_heuristic_run_time.sort(reverse=True)
    print(size, " max run time: ", subset_heuristic_run_time[:3])

print()
print()
print('total time taken : ', time.time() - start_time_overall)

40  mean time limit:  0.023715402881304422
40  max run time:  [0.06804275512695312, 0.06694149971008301, 0.06634283065795898]
60  mean time limit:  0.03751540382703145
60  max run time:  [0.10289740562438965, 0.09978389739990234, 0.0997157096862793]
80  mean time limit:  0.052524661322434746
80  max run time:  [0.1327834129333496, 0.12763047218322754, 0.12621641159057617]
100  mean time limit:  0.0711213626464208
100  max run time:  [0.34988903999328613, 0.159027099609375, 0.15554022789001465]
120  mean time limit:  0.09438423921664556
120  max run time:  [0.23330974578857422, 0.22350215911865234, 0.21761274337768555]


total time taken :  675.810308933258
